## Prediction Jupyter Notebook For Direct Interactivity

In [2]:
from ipywidgets import interact, Dropdown, IntSlider, FloatSlider
from ipywidgets.embed import embed_minimal_html
import pandas as pd
import joblib

# Load trained model
model = joblib.load("../../models/price_by_sub_area_lat_lon_model.pkl")

# Load dataset for dropdown options
X_train = pd.read_csv("../../data/processed/X_train.csv")


# Interactive function
def price_predictor(suburb, area, latitude, longitude):
    df = pd.DataFrame(
        {
            "Suburb": [suburb],
            "BuildingArea": [area],
            "Latitude": [latitude],
            "Longitude": [longitude],
        }
    )

    df = df[model.feature_names_in_]  # Ensure correct feature order
    prediction = model.predict(df).round(2)[0]

    return f"Predicted House Price: ${prediction}"


# Run interactive widget
widget = interact(
    price_predictor,
    suburb=Dropdown(options=sorted(X_train["Suburb"].unique()), description="Suburb:"),
    area=IntSlider(
        min=X_train["BuildingArea"].min(),
        max=X_train["BuildingArea"].max(),
        value=X_train["BuildingArea"].mean(),
        description="BuildingArea:",
    ),
    latitude=FloatSlider(
        min=X_train["Latitude"].min(),
        max=X_train["Latitude"].max(),
        step=0.01,
        value=X_train["Latitude"].mean(),
        description="Latitude:",
    ),
    longitude=FloatSlider(
        min=X_train["Longitude"].min(),
        max=X_train["Longitude"].max(),
        step=0.01,
        value=X_train["Longitude"].mean(),
        description="Longitude:",
    ),
)

embed_minimal_html(
    "widget_snapshot.html", views=[widget.widget], title="Interactive Widget"
)

interactive(children=(Dropdown(description='Suburb:', options=('Abbotsford', 'Aberfeldie', 'Airport West', 'Al…